In [ ]:
import json

# Load TopoJSON
with open('PUMA_or_Subborough.topo.json') as f:
    topojson_data = json.load(f)

with open('visualization.vl.json') as f:
    json_data = json.load(f)["data"]["values"]

# Map GeoID to housing problem values
value_mapping = {str(item["GeoID"]): item["Value"] for item in json_data}

# Extract transformation parameters for coordinates
scale = topojson_data['transform']['scale']
translate = topojson_data['transform']['translate']

# Function to decode arcs to coordinates
def decode_arc(arc_index, arcs_data, scale, translate):
    coords = []
    x, y = 0, 0  # Initialize starting coordinates
    arc = arcs_data[arc_index if arc_index >= 0 else ~arc_index]  # Handle reversed arcs

    for dx, dy in (arc if arc_index >= 0 else reversed(arc)):
        x += dx
        y += dy
        coords.append([(x * scale[0]) + translate[0], (y * scale[1]) + translate[1]])
    
    return coords

# Function to decode a list of arcs, for Polygons and MultiPolygons
def decode_arcs(arcs, arcs_data, scale, translate):
    return [decode_arc(arc, arcs_data, scale, translate) for arc in arcs]

# Convert TopoJSON to GeoJSON with absolute coordinates
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

# Process geometries in TopoJSON objects
for obj in topojson_data['objects'].values():
    for geom in obj['geometries']:
        properties = geom['properties']
        geo_id = geom.get('id')
        
        # Add the housingProblemValue if GeoID is found
        if geo_id in value_mapping:
            properties['housingProblemValue'] = value_mapping[geo_id]
        
        # Decode the arcs for geometry
        if geom['type'] == 'Polygon':
            coordinates = [decode_arcs(ring, topojson_data['arcs'], scale, translate) for ring in geom['arcs']]
        elif geom['type'] == 'MultiPolygon':
            coordinates = [[decode_arcs(ring, topojson_data['arcs'], scale, translate) for ring in poly] for poly in geom['arcs']]
        else:
            continue  # Unsupported geometry type
        
        # Add feature to GeoJSON
        geojson_data['features'].append({
            "type": "Feature",
            "geometry": {
                "type": geom['type'],
                "coordinates": coordinates
            },
            "properties": properties
        })

# Save the final GeoJSON with decoded coordinates
output_geojson_path = '/mnt/data/PUMA_or_Subborough_with_housingProblemValue_complete.geo.json'
with open(output_geojson_path, 'w') as f:
    json.dump(geojson_data, f)

output_geojson_path


TypeError: list indices must be integers or slices, not list